In [96]:
from selenium import webdriver
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import csv
import json
from collections import Counter
import jwt
import datetime
import requests
from lxml import html
import re
import dask.dataframe as dd # pip install dask

NaN = np.nan


headers = {
    'User-Agent': 'Mie Damgaard',
    'mail' : 'rks516@alumni.ku.dk'
}

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

In [97]:
POST_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
STRAF22_df = pd.read_csv('CSV_input/STRAF22.csv', encoding='ISO-8859-1', delimiter=';', header=None)
INDKF122_df = pd.read_csv('CSV_input/kommuneindkomst.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AREALDK_df = pd.read_csv('CSV_input/AREALDK.csv', encoding='ISO-8859-1', delimiter=';', header=None)
FOLK1E_df = pd.read_csv('CSV_input/FOLK1E.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AFSTB4_df = pd.read_csv('CSV_input/AFSTB4.csv', encoding='ISO-8859-1', delimiter=';', header=None)

In [98]:
#SKIFTE TIL FÆRRE DECIMALER
my_dataframes = [POST_df, STRAF22_df, INDKF122_df, AREALDK_df, FOLK1E_df, AFSTB4_df]

def change_to_32(dataframe):
    dataframe = dataframe.astype({col: 'float32' for col in dataframe.select_dtypes(include='float64').columns})
    dataframe = dataframe.astype({col: 'int32' for col in dataframe.select_dtypes(include='int64').columns})
    return dataframe

my_dataframes = [change_to_32(df) for df in my_dataframes]
postnummer_df, STRAF22_df, INDKF122_df, AREALDK_df, FOLK1E_df, AFSTB4_df = my_dataframes

In [99]:
#STRAF22_df
STRAF22_df = pd.read_csv('CSV_input/STRAF22.csv', encoding='ISO-8859-1', delimiter=';', header=None)
STRAF22_df.columns = ['lovovertrædelse', 'år', 'Kommune', 'antal anmeldelser', 'antal sigtelser' ]
STRAF22_df = STRAF22_df.drop(columns=['lovovertrædelse', 'år'])
STRAF22_df.drop(STRAF22_df[STRAF22_df['Kommune'].isin(['Region Hovedstaden', 'Region Sjælland', 'Region Midtjylland', 'Uoplyst kommune', 'Region Syddanmark', 'Region Nordjylland'])].index, inplace=True)
STRAF22_df

,Kommune,antal anmeldelser,antal sigtelser
0,København,93437,31358
1,Frederiksberg,8091,1770
2,Dragør,618,167
3,Tårnby,6775,2966
4,Albertslund,2572,879
...,...,...,...
94,Morsø,704,344
95,Rebild,969,365
96,Thisted,1710,883
97,Vesthimmerlands,2489,1485


In [100]:
#INDKF122_df
INDKF122_df = pd.read_csv('CSV_input/kommuneindkomst.csv', encoding='ISO-8859-1', delimiter=';', header=None)
INDKF122_df = INDKF122_df.iloc[:, -2:]
INDKF122_df.columns = ['Kommune', 'Familiernes gennemsnitlige indkomst indkomst'] 

In [101]:
#AREALDK
AREALDK_df = pd.read_csv('CSV_input/AREALDK.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AREALDK_df = AREALDK_df.iloc[:, -2:]
AREALDK_df.columns = ['Kommune', 'm2 parker, sportsanlæg og rekreative omr pr indbygger']

In [102]:
#FOLK1E_df
FOLK1E_df = pd.read_csv('CSV_input/FOLK1E.csv', encoding='ISO-8859-1', delimiter=';', header=None)
FOLK1E_df.columns = ['tid' ,'Kommune','indbyggertal', 'personer med dansk oprindelse', 'Indvandrere fra vestlige lande', 'Indvandrere fra ikke vestlige lande', 'Efterkommere fra vestlige lande', 'Efterkommere fra ikke vestlige lande']
FOLK1E_df = FOLK1E_df.apply(lambda x: x.astype('float32') if np.issubdtype(x.dtype, np.number) else x)
FOLK1E_df['andel_dansk_opr']=FOLK1E_df['personer med dansk oprindelse']/FOLK1E_df['indbyggertal']
FOLK1E_df['andel_vestlig_opr']=(FOLK1E_df['Indvandrere fra vestlige lande'] + FOLK1E_df['Efterkommere fra vestlige lande'])/FOLK1E_df['indbyggertal']
FOLK1E_df['andel_ikke_vestlig_opr']=(FOLK1E_df['Indvandrere fra ikke vestlige lande'] + FOLK1E_df['Efterkommere fra ikke vestlige lande'])/FOLK1E_df['indbyggertal']
FOLK1E_df = FOLK1E_df.drop(columns = ['tid', 'personer med dansk oprindelse', 'Indvandrere fra vestlige lande', 'Indvandrere fra ikke vestlige lande', 'Efterkommere fra vestlige lande', 'Efterkommere fra ikke vestlige lande'])

In [103]:
#AFSTB4_df
AFSTB4_df = pd.read_csv('CSV_input/AFSTB4.csv', encoding='ISO-8859-1', delimiter=';', header=None)
AFSTB4_df.columns = ['Kommune', 'Gennemsnitlig_pendligafstand']

In [104]:
merged_df = pd.merge(STRAF22_df, INDKF122_df, on='Kommune', how = 'left')
merged_df = pd.merge(merged_df, AREALDK_df, on='Kommune', how ='left')
merged_df = pd.merge(merged_df, FOLK1E_df, on='Kommune', how ='left')
merged_df = pd.merge(merged_df, AFSTB4_df, on='Kommune', how ='left')
merged_df = merged_df[~merged_df['Kommune'].isin(['Uoplyst kommune', 'Christiansø'])]
merged_df['Kommune'] = merged_df['Kommune'] + ' kommune'
merged_df['Kommune'] = merged_df['Kommune'].replace('København kommune', 'Københavns kommune')
merged_df

,Kommune,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand
0,Københavns kommune,93437,31358,369458.0,16.0,661831.0,0.717574,0.116602,0.165823,12.6
1,Frederiksberg kommune,8091,1770,451283.0,9.8,105196.0,0.796589,0.091334,0.112077,12.4
2,Dragør kommune,618,167,612857.0,69.8,14457.0,0.911877,0.043093,0.045030,18.1
3,Tårnby kommune,6775,2966,453360.0,29.0,43957.0,0.840526,0.047000,0.112474,13.6
4,Albertslund kommune,2572,879,371813.0,44.4,27545.0,0.668760,0.046179,0.285061,13.6
...,...,...,...,...,...,...,...,...,...,...
94,Morsø kommune,704,344,388518.0,105.1,19611.0,0.928153,0.028300,0.043547,19.6
95,Rebild kommune,969,365,486463.0,84.0,30962.0,0.934662,0.025224,0.040114,30.2
96,Thisted kommune,1710,883,404770.0,85.1,42910.0,0.903193,0.045747,0.051060,22.5
97,Vesthimmerlands kommune,2489,1485,403184.0,169.6,35864.0,0.893486,0.048991,0.057523,25.7


In [119]:
merged_df.to_csv('CSV_output/DST.csv', index=False)

In [ ]:
#POSTNUMMER
POST_df = pd.read_csv('CSV_input/postnumre.csv', encoding='ISO-8859-1', delimiter=';', header=None)
POST_df = POST_df.iloc[:, 2] # Gem kolonne med string: 'postnummer, bynavn, kommune'
POST_df = POST_df.str.extract(r'(\d+)\s+(.+)\s\((.+)\)')  # Split kolonnen til 'Postal code', 'By_fra_DST', 'Kommune_fra_DST'
#GAMLE:      POST_df.columns = ['Postal code', 'By_fra_DST', 'Kommune_fra_DST']
#NYE:
POST_df.columns = ['Postal code', 'By_fra_DST', 'Kommune_fra_DST']
POST_df = POST_df.dropna(how='all')
POST_df['Postal code'] = POST_df['Postal code'].astype('int32')
#POST_df